Archivos:
https://drive.google.com/file/d/1Z808iMAlYU8GEUmhb5O58rpFoYEgIyYH/view?usp=sharing

https://drive.google.com/file/d/17peZoammCnbLdeht-IDql9oSzHFbsGIe/view?usp=sharing

https://drive.google.com/file/d/1JZ6BwKhQdUT5KPbAEixkGQWmlfEKhcPd/view?usp=sharing

# Proyecto 03 - Procesamiento del Lenguaje Natural

## Base de datos: The Multilingual Amazon Reviews Corpus

**Recuerda descargar el data de [aquí](https://github.com/kang205/SASRec). Es un archivo .zip que contiene tres documentos. Más información sobre el data [aquí](https://registry.opendata.aws/amazon-reviews-ml/). Es importante que tengas en cuenta la [licencia](https://docs.opendata.aws/amazon-reviews-ml/license.txt) de este data.**

### Exploración de datos y Procesamiento del Lenguaje Natural

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Considera que hasta que no hayas aplicado las herramientas de Procesamiento del Lenguaje Natural vistas, será difícil completar este análisis. Elige preguntas que creas que puedas responder con este data. Por ejemplo, ¿qué palabras están asociadas a calificaciones positivas y qué palabras a calificaciones negativas?

### Machine Learning

Implementa un modelo que, dada la crítica de un producto, asigne la cantidad de estrellas correspondiente. **Para pensar**: ¿es un problema de Clasificación o de Regresión?

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.
1. Intenta responder la pregunta: ¿Qué información está usando el modelo para predecir?

**Recomendación:** si no te resulta conveniente trabajar en español con NLTK, te recomendamos que explores la librería [spaCy](https://spacy.io/).

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Valdrá la pena convertir el problema de Machine Learning en un problema binario? Es decir, asignar únicamente las etiquetas Positiva y Negativa a cada crítica y hacer un modelo que, en lugar de predecir las estrellas, prediga esa etiqueta. Pensar en qué situación puede ser útil. ¿Esperas que el desempeño sea mejor o peor?
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
# Vincular la cuenta de Google Drive donde están almacenados los archivos
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data = pd.read_json("/content/gdrive/MyDrive/dataset_es_train.json", lines= True)
data.head()

In [ ]:
len(data['product_id'].unique())

In [ ]:
len(data)

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
# Observo los tipos de tados por variable
data.dtypes

In [ ]:
# Calculo los estadísticos principales (sólo tenemos una variable numérica)
data.describe()

In [ ]:
# CALCULO MISSING

a = data[['review_body','review_title', 'product_category', 'stars']].isnull().sum(axis=0)
b = round(a/data.shape[0]*100,2)

missing_df = pd.DataFrame({'missing_totales' : a, 'missing_freq' : b})

missing_df[missing_df['missing_totales']>0]

La base de datos no presenta valores faltantes

## Distribucion de las reviews

In [ ]:
plt.rc("figure", figsize=(5, 5))

# Grafico
ax = sns.countplot(data = data, x = 'stars', orient="v", palette ='Set3')

plt.title('Cantidad Reviews por Stars')
plt.xlabel('Star')
plt.ylabel('Cantidad')


for p in ax.patches:
    ax.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
#ax.yaxis.set_ticks(np.linspace(0, len_star, 10))

plt.show()

Con el presente gráfico podemos observar que cada una de nuestra clases se encuentran balanceadas, presentando 40 mil instancias por cantidad de estrellas.



In [ ]:
plt.rc("figure", figsize=(20, 10))

# Grafico
ax = sns.countplot(data = data, x = 'product_category', order= data['product_category'].value_counts().index, orient="v", palette ='Set3')
plt.setp(ax.get_xticklabels(), rotation=90)

plt.title('Cantidad Reviews por Categoría del Procuto')
plt.xlabel('Star')
plt.ylabel('Cantidad')


for p in ax.patches:
    ax.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
plt.show()

In [ ]:
#Preparo los datos

N_TOP = 10

# Preparo los datos para graficar

count = data.groupby(['stars','product_category'], as_index=False)['language'].count()
count_max = count.sort_values(['stars', 'language'], ascending=False).groupby('stars').head(N_TOP)

# Renombro
count_max = count_max.rename(columns = {'language': 'Total'}, inplace = False)

# Joineo con la data original para quedarme sólo con los barrios que están en el top
data_grap = data.merge(count_max, how='left', on=['stars','product_category'])

#Reemplazo nuelos por 0
data_grap = data_grap.fillna(0)

# Reemplazo por OTROS para graficar
data_grap['product_category']= np.where(data_grap['Total'] == 0,'OTROS', data_grap['product_category'])

In [ ]:
data_grap.head(2)

In [ ]:
# Grafico

# 1 ESTRELLA #######################################
plt.figure(figsize=(11,14))
plt.subplot(5, 1, 1)

g1 = sns.countplot(data = data_grap[data_grap['stars']==1], x='product_category', 
                   order = data_grap[data_grap['stars']==1]['product_category'].value_counts().index.drop('OTROS').insert(N_TOP+1, 'OTROS'), 
                   orient="v", palette ='Set3')
plt.setp(g1.get_xticklabels(), rotation=90)

plt.title('Cantidad de Productos con 1 Estrella')
plt.xlabel('1 Estrella')
plt.ylabel('Cantidad')

for p in g1.patches:
    g1.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))

# 2 ESTRELLAS #######################################
plt.figure(figsize=(11,14))
plt.subplot(5, 1, 2)

g2 = sns.countplot(data = data_grap[data_grap['stars']==2], x='product_category', 
                   order = data_grap[data_grap['stars']==2]['product_category'].value_counts().index.drop('OTROS').insert(N_TOP+1, 'OTROS'), 
                   orient="v", palette ='Set3')
plt.setp(g2.get_xticklabels(), rotation=90)

plt.title('Cantidad de Productos con 2 Estrellas')
plt.xlabel('2 Estrellas')
plt.ylabel('Cantidad')

for p in g2.patches:
    g2.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
# 3 ESTRELLAS #######################################
plt.figure(figsize=(11,14))
plt.subplot(5, 1, 3)

g3 = sns.countplot(data = data_grap[data_grap['stars']==3], x='product_category', 
                   order = data_grap[data_grap['stars']==3]['product_category'].value_counts().index.drop('OTROS').insert(N_TOP+1, 'OTROS'), 
                   orient="v", palette ='Set3')
plt.setp(g3.get_xticklabels(), rotation=90)

plt.title('Cantidad de Productos con 3 Estrellas')
plt.xlabel('3 Estrellas')
plt.ylabel('Cantidad')

for p in g3.patches:
    g3.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
    
# 4 ESTRELLAS #######################################
plt.figure(figsize=(11,14))
plt.subplot(5, 1, 4)

g4 = sns.countplot(data = data_grap[data_grap['stars']==4], x='product_category', 
                   order = data_grap[data_grap['stars']==4]['product_category'].value_counts().index.drop('OTROS').insert(N_TOP+1, 'OTROS'), 
                   orient="v", palette ='Set3')
plt.setp(g4.get_xticklabels(), rotation=90)

plt.title('Cantidad de Productos con 4 Estrellas')
plt.xlabel('4 Estrellas')
plt.ylabel('Cantidad')

for p in g4.patches:
    g4.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))

# 5 ESTRELLAS #######################################
plt.figure(figsize=(11,14))
plt.subplot(5, 1, 5)

g5 = sns.countplot(data = data_grap[data_grap['stars']==5], x='product_category', 
                   order = data_grap[data_grap['stars']==5]['product_category'].value_counts().index.drop('OTROS').insert(N_TOP+1, 'OTROS'), 
                   orient="v", palette ='Set3')
plt.setp(g5.get_xticklabels(), rotation=90)

plt.title('Cantidad de Productos con 5 Estrellas')
plt.xlabel('5 Estrellas')
plt.ylabel('Cantidad')

for p in g5.patches:
    g5.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
#plt.savefig('snscounter.pdf')


Los gráicos nos muestran que los tipos de productos se distribuyen casi uniformemente en cada una de las clases a predecir, presentando en todos los casos las mismas 10 principales categorias por estrella.

* Home
* Wireless
* Toy
* Sports
* PC
* Home Improvement
* Electronics
* Book
* Beauty
* Kitchen

**CONSIDERACIONES**

De nuestro dataset original, únicamente utilizaremos las variables de review_body, review_title y star ya que son las únicas que nos aportan información para la predicción.

Originalmente se consideró que la variable product_category podría brindarnos información para nuestro problema, al comprobar que la cantidad de instancias por tipos de productos está uniformemente distribuido en cada una de las estrellas, entendemos que no podremos extraer un diferencial de dicha variable. Posiblemente esto suceda porque estamos utilizando un dataset para fines académicos cuyas muestras fueron seleccionadas con esta distribución intencionalmente.

Finalmente, el tipo de problema que estamos abordando no requiere de las transformaciones clásicas de datos tales como:

* Detección y eliminación de outliers
* Encoding
* Imnputación de valores faltantes
* Escalado de datos
* Generación de nuevas variables predictoras/reducción de dimensionalidad (SVD/PCA).

Sin embargo, en un apartado posterior se realizarán las transformaciones encesarias para convertir nuestra variable alfanumérica en numérica y así poder ser consumible por los modelos de machine learning a aplicar.

Nuevas Variables:
A continuación se crearán las siguientes variables:

* star_calif: La cuál tomará valor de 0 (Negativo) si la variable star es menor o igual a 3 estrellas y 1 (Positivo) si es mayor.
* review_all: La misma concatenará la información provistar por las variables * review_title y review_body

In [ ]:
# Creamos la variable stars_calif para análisis de sentimiento
data['stars_calif'] = [1 if data['stars'][i]> 3 else 0 for i in data.index]

In [ ]:
# Creo la variable 'review_all', que es una concatenación de 'review_title' y 'review_body'
data['review_all']=[(str(data['review_title'][i])+" "+str(data['review_body'][i])) for i in data.index]

In [ ]:
data['review_all'][5]

## Submuestreo de Clases:


Para todo el trabajo que se presentará a continuación, la capacidad de procesamiento local con la que se cuenta y la provista por Colab, son insuficientes para poder procesar el dataset de train completo con sus 200 mil instancias. Cabe aclarar al respecto, que se han agotado los intentos por procesar el dataset completo y en todos los casos por tiempos de ejecución o límite de la memoria ram, colab ha interrumpido el trabajo e iniciado a cero la ejecución.

Es por lo expuesto que se ha decido realizar un submuestreo incial de 5 mil instancias por clase.

In [ ]:
# Submuestro y balanceo de clases
data_sample = data.groupby('stars')
data_sample = pd.DataFrame(data_sample.apply(lambda x: x.sample(data_sample.size().min()-35000).reset_index(drop=True))).reset_index(drop=True)

In [ ]:
data_train = data_sample.copy()

In [ ]:
data_train.shape

In [ ]:
data_train.head()

In [ ]:
plt.rc("figure", figsize=(5, 5))

# Grafico

ax = sns.countplot(data = data_train, x = 'stars', orient="v", palette ='Set3')

plt.title('Cantidad Reviews por Stars')
plt.xlabel('Star')
plt.ylabel('Cantidad')


for p in ax.patches:
    ax.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))

plt.show()

## Analisis de Sentimiento

In [ ]:
plt.rc("figure", figsize=(5, 5))

# Grafico

ax = sns.countplot(data = data_train, x = 'stars_calif', orient="v", palette ='Set3')

plt.title('Cantidad Reviews por Sentimiento')
plt.xlabel('Star')
plt.ylabel('Cantidad')


for p in ax.patches:
    ax.annotate('{:,.0f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
plt.show()

## Normalizacion 

In [ ]:
!python -m spacy download es_core_news_md

In [ ]:
import spacy
#import es_core_news_sm
import es_core_news_md
from spacy.lang.es.stop_words import STOP_WORDS

import nltk
from nltk import Tree
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.toktok import ToktokTokenizer
#from nltk.corpus import stopwords

from wordcloud import WordCloud, STOPWORDS
import string
from collections import Counter
import unicodedata
import re
import itertools

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

nlp = es_core_news_md.load()
tokenizer = ToktokTokenizer()
stopword_list=  nltk.corpus.stopwords.words('spanish')
stopwords = spacy.lang.es.stop_words.STOP_WORDS
punct = string.punctuation

In [ ]:
NLP_train = data_sample.copy()

NLP_train_some = NLP_train[['review_all']][0:10].copy()

In [ ]:
docex=nlp(NLP_train['review_all'][3])

# print column headers
print('{:15} | {:15} | {:8} | {:20} | {:11} | {:8} | {:8} | {:8} | '.format(
    'TEXT','LEMMA_','POS_','HEAD','DEP_','SHAPE_','IS_ALPHA','IS_STOP'))

# print various SpaCy POS attributes
for token in docex:
    print('{:15} | {:15} | {:8} | {:20} | {:11} | {:8} | {:8} | {:8} |'.format(
          token.text, token.lemma_, token.pos_, token.head.text, token.dep_
        , token.shape_, token.is_alpha, token.is_stop))

In [ ]:
# Similaridad de Coseno entre reviews
docex_1=nlp(NLP_train['review_all'][0])
docex_2=nlp(NLP_train['review_all'][1])

print(docex_1)
print(docex_2)
print(docex_1.similarity(docex_2))

In [ ]:
# Similaridad de Coseno entre tokens
token1 = docex_1[0]
token2 = docex_2[0]

print(token1)
print(token2)
print(token1.similarity(token2))

In [ ]:
# Gráfico de dependencias de una reviw
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_
[to_nltk_tree(sent.root).pretty_print() for sent in docex.sents]

In [ ]:
# Vector con Word2Vec
print(token2.vector)

In [ ]:
tokens = []
lemma = []
pos = []

for doc in nlp.pipe(NLP_train_some['review_all'].astype('unicode').values, batch_size=50):
    if doc.is_parsed:
        # Token
        tokens.append([n.text for n in doc])
        # Token lematizado y en minúscula
        lemma.append([n.lemma_.lower() for n in doc])
        # Part of speach
        pos.append([n.pos_ for n in doc])
    else:
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

NLP_train_some['review_tokens'] = tokens
NLP_train_some['review_lemma'] = lemma
NLP_train_some['review_pos'] = pos

In [ ]:
NLP_train_some

**Función de Normalización de texto:**


En dicha función, realizaremos el siguiente proceso:

Se tokenizarán las reviws. La unidad semántica elegida para la tokenización serán palabras.
Se eliminarán las palabras de menos de 2 caracteres.
Aplicaremos Lematización, en donde agruparemos a las palabras por su raíz y el rol que cumple en el texto.
Eliminaremos las palabras que el lematizador identifique como Pronombre.
Llevaremos todas las palabras a minúscula.
Eliminaremos las palabras que no transmiten información (Stopwords).
Aplicaremos Expresiones Regulares (Regex) para eliminar cualquier patrón en el texto que no nos aporte información.

In [ ]:
stopwords.remove('no')

In [ ]:
def dataCleaning(sentence):
    doc = nlp(sentence)
    tokens = []
    for token in doc:
        if len(token)>1: #si el token tiene más de 1 caracter
            # Forma base del token, sin sufijos de flexión. Y lo pasamos a minuscula.
            if token.lemma_ != '-PRON-':
              temp = token.lemma_.lower()
              tokens.append(temp)
              clean_tokens = []
              # Quitamos stopswords
              for token in tokens:
                  #if token not in punct and token not in stopwords:
                  if token not in stopwords:
                      clean_tokens.append(token)
    return clean_tokens

def remove_accented_chars(text):
    # Removemos los caracteres especiales
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Eliminamos cualquier caracter que no sen los siguientes: a-z A-Z 0-9   
    pattern = r'[^a-zA-Z0-9\s]' 
    text = re.sub(pattern, '', text)
     
    return text


Aplicamos nuestra función de normalización al texto

In [ ]:
titular_list_clean=[]

i=0
titular_clean=[]
for titular in NLP_train['review_all']:
    titular=remove_accented_chars(str(titular))
    titular_clean=dataCleaning(titular)
    titular_list_clean.append(titular_clean)
    i=+1

In [ ]:
data_train['Review_Cleaning'] = titular_list_clean

In [ ]:
data_train.head()

In [ ]:
data_review_positivos = data_train[data_train['stars_calif']==1]
review_positivos = data_review_positivos['Review_Cleaning'].copy()
review_positivos.head(2)

In [ ]:
data_review_negativos = data_train[data_train['stars_calif']==0]
review_negativos = data_review_negativos['Review_Cleaning'].copy()
review_negativos.head(2)

In [ ]:
todos_titulares_palabras_pos = list(itertools.chain(*review_positivos))
todos_titulares_palabras_neg = list(itertools.chain(*review_negativos))

In [ ]:
count_pos=Counter(todos_titulares_palabras_pos)
count_neg=Counter(todos_titulares_palabras_neg)

bigrams_series_pos = (pd.Series(nltk.ngrams(todos_titulares_palabras_pos, 2)).value_counts())[:20]
trigrams_series_pos = (pd.Series(nltk.ngrams(todos_titulares_palabras_pos, 3)).value_counts())[:20]
bigrams_series_neg = (pd.Series(nltk.ngrams(todos_titulares_palabras_neg, 2)).value_counts())[:20]
trigrams_series_neg = (pd.Series(nltk.ngrams(todos_titulares_palabras_neg, 3)).value_counts())[:20]

In [ ]:
# Plot top 50 most frequently
common_words = [word[0] for word in count_pos.most_common(50)]
common_counts = [word[1] for word in count_pos.most_common(50)]
#plt.style.use('dark_background')

fig, ax = plt.subplots(figsize=(15, 10))
sns.barplot(x=common_words, y=common_counts, ax=ax)
plt.title('Top Palabras Frecuentes Reviews Positivos')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

In [ ]:
bigrams_series_pos.sort_values().plot.barh(width=.9, colormap='Paired', figsize=(12, 8))
plt.title('Top Bigramas Frecuentes Reviews Positivos')

plt.show()

In [ ]:
trigrams_series_pos.sort_values().plot.barh(width=.9, colormap='Paired',  figsize=(12, 8))
plt.title('Top Trigramas Frecuentes Reviews Positivos')

plt.show()

In [ ]:
plt.rc("figure", figsize=(15, 15))

wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(str(common_words))
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('Word Cloud Reviews Positivos')
plt.axis("off")
plt.show()

In [ ]:
# Plot top 50 most frequently
common_words = [word[0] for word in count_neg.most_common(50)]
common_counts = [word[1] for word in count_neg.most_common(50)]
#plt.style.use('dark_background')

fig, ax = plt.subplots(figsize=(15, 10))
sns.barplot(x=common_words, y=common_counts, ax=ax)
plt.title('Top Palabras Frecuentes Reviews Negativos')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

In [ ]:
bigrams_series_neg.sort_values().plot.barh(width=.9, colormap='Paired', figsize=(12, 8))
plt.title('Top Bigramas Frecuentes Reviews Negativos')

plt.show()

In [ ]:
trigrams_series_neg.sort_values().plot.barh(width=.9, colormap='Paired', figsize=(12, 8))
plt.title('Top Bigramas Frecuentes Reviews Negativos')

plt.show()

In [ ]:
plt.rc("figure", figsize=(15, 15))

wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(str(common_words))
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('Word Cloud Reviews Negativos')
plt.axis("off")
plt.show()

## 3. Transformación de Datos


Seguida de la Normalización de nuestro texto, debemos Vectorizarlo. Es decir, reemplazar a cada instancia por un vector de números que represente a cada uno de los tokens de dicha instancia.

En este apartado trabajaremos con dos tipos de vectorizaciones:

*** Count Vectorizer:** Convierte la columna de texto en una matriz en la que cada palabra es una columna cuyo valor es el número de veces que dicha palabra aparece en cada review.

*** TF-IDF:** Term Frequency-Inverse Document Frequency: Busca puntuaciones de frecuencia de palabras que tratan de resaltar las palabras que son más interesantes, por ejemplo, frecuentes en un documento pero no en todos los documentos. El TfidfVectorizer tokenizará documentos, aprenderá el vocabulario y las ponderaciones inversas de frecuencia de documentos.


A continuación, probaremos optimizando el rendimiento de un posterior modelo modificando los valores de los parámetros de countvectorizer y tfidfvectorizer respectivamente.

In [ ]:
titular_list_clean=[]

i=0
titular_clean=[]
for titular in data_train['review_all']:
    titular=remove_accented_chars(str(titular))
    titular_clean=dataCleaning(titular)
    titular_clean=' '.join(titular_clean)
    titular_list_clean.append(titular_clean)
    i=+1

In [ ]:
result = pd.Series(titular_list_clean)

In [ ]:
result

In [ ]:
len(result)

In [ ]:
result.to_csv('result.csv', header=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

Parámetros de Count Vectorizer



In [ ]:
max_features = [100,200,300,500,700,1000,1300,1500] #No se prueban más features porque agota toda la memoria ram de colab
labels = [1, 2, 3, 4, 5]
training_auc = [] 
testing_auc = []


for max_feature in max_features:
    # Vectorizo con count vectorizer
    cvectorizer = CountVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore', max_features=max_feature)
    matriz_titulos_count_vectorizer = cvectorizer.fit_transform(result)

    X = matriz_titulos_count_vectorizer.toarray()
    Y = data_train['stars']

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

    # Definir el modelo y entreno
    clf = LGBMClassifier().fit(X_train,Y_train)
    
    # Binarize ytest with shape (n_samples, n_classes)
    Y_test = label_binarize(Y_test, classes=labels)

    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = clf.predict(X_train)
    y_train_preds = label_binarize(y_train_pred, classes=labels)
    train_roc_auc_score = roc_auc_score(Y_train,y_train_preds, multi_class='ovr')
    
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = clf.predict(X_test)
    y_test_preds = label_binarize(y_test_pred, classes=labels)
    test_roc_auc_score = roc_auc_score(Y_test,y_test_preds, multi_class='ovr') 
    
    # Agregar la información a las listas
    training_auc.append(train_roc_auc_score)
    testing_auc.append(test_roc_auc_score)

In [ ]:
plt.rc("figure", figsize=(6, 6))
plt.plot(max_features, training_auc, color='blue', label='Training Roc AUC Score')
plt.plot(max_features, testing_auc, color='green', label='Testing Roc AUC Score')
plt.xlabel('Max Features')
plt.ylabel('Roc AUC Score')
plt.title('Hyperparameter Tuning', pad=15, size=15)
plt.legend()
#plt.savefig('error.png') 

In [ ]:
# Submuestro y balanceo de clases
data_sample_min = data_train.groupby('stars')
data_sample_min = pd.DataFrame(data_sample_min.apply(lambda x: x.sample(data_sample_min.size().min()-4500).reset_index(drop=True))).reset_index(drop=True)

In [ ]:
data_sample_min.shape


In [ ]:
data_sample_min.head(2)

In [ ]:
titular_list_clean=[]

i=0
titular_clean=[]
for titular in data_sample_min['review_all']:
    titular=remove_accented_chars(str(titular))
    titular_clean=dataCleaning(titular)
    titular_clean=' '.join(titular_clean)
    titular_list_clean.append(titular_clean)
    i=+1

In [ ]:
result_data_sample_min = pd.Series(titular_list_clean)


In [ ]:
result_data_sample_min

In [ ]:
len(result_data_sample_min)

In [ ]:
type(result_data_sample_min)


In [ ]:
# Al construir el vocabulario, ignore los términos que tienen una frecuencia de documento estrictamente más alta que el umbral dado

max_dfs  = [0.8,0.9,1]
labels = [1, 2, 3, 4, 5]
training_auc = [] 
testing_auc = []


for max_df in max_dfs:
    # Vectorizo con count vectorizer
    cvectorizer = CountVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore', max_df=max_df)
    matriz_titulos_count_vectorizer = cvectorizer.fit_transform(result_data_sample_min)

    X = matriz_titulos_count_vectorizer.toarray()
    Y = data_sample_min['stars']

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

    # Definir el modelo y entreno
    clf = LGBMClassifier().fit(X_train,Y_train)
    
    # Binarize ytest with shape (n_samples, n_classes)
    Y_test = label_binarize(Y_test, classes=labels)

    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = clf.predict(X_train)
    y_train_preds = label_binarize(y_train_pred, classes=labels)
    train_roc_auc_score = roc_auc_score(Y_train,y_train_preds, multi_class='ovr')
    
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = clf.predict(X_test)
    y_test_preds = label_binarize(y_test_pred, classes=labels)
    test_roc_auc_score = roc_auc_score(Y_test,y_test_preds, multi_class='ovr') 
    
    # Agregar la información a las listas
    training_auc.append(train_roc_auc_score)
    testing_auc.append(test_roc_auc_score)


In [ ]:
plt.rc("figure", figsize=(6, 6))
plt.plot(max_dfs, training_auc, color='blue', label='Training Roc AUC Score')
plt.plot(max_dfs, testing_auc, color='green', label='Testing Roc AUC Score')
plt.xlabel('Max Df')
plt.ylabel('Roc AUC Score')
plt.title('Hyperparameter Tuning', pad=15, size=15)
plt.legend()
plt.show()

Parámetros de TF-IDF Vectorizer

In [ ]:
max_features = [100,200,300,500,700,1000,1300,1500]
labels = [1, 2, 3, 4, 5]
training_auc = [] 
testing_auc = []


for max_feature in max_features:
    # Vectorizo con count vectorizer
    tvectorizer = TfidfVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore', max_features=max_feature)
    matriz_titulos_count_vectorizer = tvectorizer.fit_transform(result)

    X = matriz_titulos_count_vectorizer.toarray()
    Y = data_train['stars']

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

    # Definir el modelo y entreno
    clf = LGBMClassifier().fit(X_train,Y_train)
    
    # Binarize ytest with shape (n_samples, n_classes)
    Y_test = label_binarize(Y_test, classes=labels)

    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = clf.predict(X_train)
    y_train_preds = label_binarize(y_train_pred, classes=labels)
    train_roc_auc_score = roc_auc_score(Y_train,y_train_preds, multi_class='ovr')
    
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = clf.predict(X_test)
    y_test_preds = label_binarize(y_test_pred, classes=labels)
    test_roc_auc_score = roc_auc_score(Y_test,y_test_preds, multi_class='ovr') 
    
    # Agregar la información a las listas
    training_auc.append(train_roc_auc_score)
    testing_auc.append(test_roc_auc_score)

In [ ]:
plt.rc("figure", figsize=(6, 6))
plt.plot(max_features, training_auc, color='blue', label='Training Roc AUC Score')
plt.plot(max_features, testing_auc, color='green', label='Testing Roc AUC Score')
plt.xlabel('Max Features')
plt.ylabel('Roc AUC Score')
plt.title('Hyperparameter Tuning', pad=15, size=15)
plt.legend()
#plt.savefig('error.png') 

In [ ]:
# Al construir el vocabulario, ignore los términos que tienen una frecuencia de documento estrictamente más alta que el umbral dado

max_dfs  = [0.7,0.8,0.9,1]
labels = [1, 2, 3, 4, 5]
training_auc = [] 
testing_auc = []


for max_df in max_dfs:
    # Vectorizo con count vectorizer
    tvectorizer = TfidfVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore', max_df=max_df)
    matriz_titulos_count_vectorizer = tvectorizer.fit_transform(result_data_sample_min)

    X = matriz_titulos_count_vectorizer.toarray()
    Y = data_sample_min['stars']

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

    # Definir el modelo y entreno
    clf = LGBMClassifier().fit(X_train,Y_train)
    
    # Binarize ytest with shape (n_samples, n_classes)
    Y_test = label_binarize(Y_test, classes=labels)

    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = clf.predict(X_train)
    y_train_preds = label_binarize(y_train_pred, classes=labels)
    train_roc_auc_score = roc_auc_score(Y_train,y_train_preds, multi_class='ovr')
    
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = clf.predict(X_test)
    y_test_preds = label_binarize(y_test_pred, classes=labels)
    test_roc_auc_score = roc_auc_score(Y_test,y_test_preds, multi_class='ovr') 
    
    # Agregar la información a las listas
    training_auc.append(train_roc_auc_score)
    testing_auc.append(test_roc_auc_score)


In [ ]:
plt.rc("figure", figsize=(6, 6))
plt.plot(max_dfs, training_auc, color='blue', label='Training Roc AUC Score')
plt.plot(max_dfs, testing_auc, color='green', label='Testing Roc AUC Score')
plt.xlabel('Max Df')
plt.ylabel('Roc AUC Score')
plt.title('Hyperparameter Tuning', pad=15, size=15)
plt.legend()
plt.show()

VECTORIZACIÓN FINAL

En ambos casos, los parámetros que mejorarían nuestra performance del modelo son similares, motivo por el cual elegiremos los mismos para ambos tipos de vectorización.

In [ ]:
# numero minimo y maximo de tokens consecutivos que se consideran
MIN_NGRAMS=1
MAX_NGRAMS=4
# cantidad maxima de docs que tienen que tener a un token para conservarlo.
MAX_DF= 0.8
max_features=1000

In [ ]:
cvectorizer = CountVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore',
                             ngram_range=(MIN_NGRAMS, MAX_NGRAMS), max_df=MAX_DF, max_features=max_features)
matriz_titulos_count_vectorizer = cvectorizer.fit_transform(result)

In [ ]:
tvectorizer = TfidfVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore',
                             ngram_range=(MIN_NGRAMS, MAX_NGRAMS), max_df=MAX_DF, max_features=max_features)
matriz_titulos_count_tvectorizer = tvectorizer.fit_transform(result)

## 4. Modelos de Machine Learning


En este apartado nos propondremos contruir un clasificador que prediga la cantidad de estrellas con la que puntuará un cliente una compra a partir de su reseña. Para ello, trabajaremos con las siguientes iteraciones:

**Iteración 1:** Se entrenará un modelo LGBM Classifier con CountVectorizer y TfidfVectorizer para probar el desempeño de ambos vectorizers y elegir el que mejor performe. Asimimo, evaluaremos la importancia de atributos y observaremos gráficamente las reglas de decisión principales. El modelo resultante de este apartado será nuestro benchmark.

**Iteración 2:** Se seleccionarán las características principales de nuestro total de features lo cual nos permitirá entrenar el modelo con más instancias. Asimismo, ee entrenarán diferentes modelos para evaluar si es necesario aplicar una técnica de boosting o podemos obtener los mismos resultados con un modelo más simple.

**Iteración 3:** Se seleccionará el mejor modelo y se aplicará Randomized Search para optimizarán hiperparámtros y evaluar la estabilidad del modelo con 5 k fold validation.

Como consideración, para evaluar el rendimiento del modelo multiclase, se utilizará la métrica con ROC Curve, la cual calcula el área bajo la curva del receptor (ROC AUC) a partir de los resultados de las predicciones, cuyo resultado será el valor promedio del score obtenido en cada una de las clases contra todas las demás.

### 1ª Iteración 

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import graphviz

In [ ]:
# COUNT VECTORIZER
X1 = matriz_titulos_count_vectorizer.toarray()
Y1 = data_train['stars']

X_train, X_test, y_train, y_test = train_test_split(X1,Y1,test_size=0.3,random_state=42,stratify=Y1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
LGBM_cl= LGBMClassifier().fit(X_train,y_train)

labels = [1, 2, 3, 4, 5]
# Binarize ytest with shape (n_samples, n_classes)
y_testb = label_binarize(y_test, classes=labels)

y_train_pred = LGBM_cl.predict(X_train)
y_test_pred = LGBM_cl.predict(X_test)

# Binarize ypreds with shape (n_samples, n_classes)
y_train_preds = label_binarize(y_train_pred, classes=labels)
y_test_preds = label_binarize(y_test_pred, classes=labels)
    
print('Modelo: LGBM_cl')
print('ROC AUC Train', roc_auc_score(y_train,y_train_preds, multi_class='ovr'))
print('ROC AUC Test', roc_auc_score(y_test,y_test_preds, multi_class='ovr'))
metrics.plot_confusion_matrix(LGBM_cl, X_test, y_test, values_format = '.0f')
plt.show()
print('\n')

In [ ]:
# TF-IDF VECTORIZER
X2 = matriz_titulos_count_tvectorizer.toarray()
Y2 = data_train['stars']

X_train, X_test, y_train, y_test = train_test_split(X2,Y2,test_size=0.3,random_state=42,stratify=Y2)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
LGBM_cl= LGBMClassifier().fit(X_train,y_train)

labels = [1, 2, 3, 4, 5]
# Binarize ytest with shape (n_samples, n_classes)
y_testb = label_binarize(y_test, classes=labels)

y_train_pred = LGBM_cl.predict(X_train)
y_test_pred = LGBM_cl.predict(X_test)

# Binarize ypreds with shape (n_samples, n_classes)
y_train_preds = label_binarize(y_train_pred, classes=labels)
y_test_preds = label_binarize(y_test_pred, classes=labels)
    
print('Modelo: LGBM_cl')
print('ROC AUC Train', roc_auc_score(y_train,y_train_preds, multi_class='ovr'))
print('ROC AUC Test', roc_auc_score(y_test,y_test_preds, multi_class='ovr'))
metrics.plot_confusion_matrix(LGBM_cl, X_test, y_test, values_format = '.0f')
plt.show()
print('\n')

Se puede observar una pequeña mejoría aplicando tf-idf, por lo que se seleccionará dicho modelo como nuestro benchmark.

In [ ]:
# Ordeno las features más importantes
thresholds = sorted(LGBM_cl.feature_importances_, reverse=True)

# Me quedo con las primeras 100
thresholds = thresholds[:100]

In [ ]:
# Evalúo el ROC AUC que obtengo incorporando de a una feature al modelo
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(LGBM_cl, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    
    # train model
    selection_model = LGBMClassifier(random_state=7)
    selection_model.fit(select_X_train, y_train)
    
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    y_preds = label_binarize(y_pred, classes=labels)
    predictions = [np.round(value) for value in y_preds]
    roc_auc = roc_auc_score(y_test,predictions, multi_class='ovr')
    print("Thresh=%.3f, n=%d, ROC AUC: %.2f%%" % (thresh, select_X_train.shape[1], roc_auc*100.0))

In [ ]:
# Genero un dataframe con las features y su importancia

atributos = tvectorizer.get_feature_names()
feat_imp = pd.DataFrame({'Atributo':atributos,'importancia':LGBM_cl.feature_importances_}).sort_values('importancia',ascending=False)
most_important_features = feat_imp[:200]

# Exporto para poder analizar
#feat_imp.to_excel('feature_importance.xlsx')

In [ ]:
feat_imp.head()

In [ ]:
N = 30
first_N = feat_imp[:N]

first_N[:30].sort_values('importancia',ascending=True).plot.barh(x='Atributo', width=.9, colormap='Paired', figsize=(12, 8))

In [ ]:
# print column headers
print('{:15} | {:15} | {:8} | {:20} | {:11} | {:8} | {:8} | {:8} | '.format('TEXT','LEMMA_','POS_','HEAD','DEP_','SHAPE_','IS_ALPHA','IS_STOP'))

for i in first_N['Atributo']:
  doc=nlp(i)
  # print various SpaCy POS attributes
  for token in doc:
    print('{:15} | {:15} | {:8} | {:20} | {:11} | {:8} | {:8} | {:8} |'.format(token.text, token.lemma_, token.pos_, token.head.text, token.dep_ , token.shape_, token.is_alpha, token.is_stop))

In [ ]:
DecisionTree_cl = DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5).fit(X_train,y_train)

labels = [1, 2, 3, 4, 5]
# Binarize ytest with shape (n_samples, n_classes)
y_testb = label_binarize(y_test, classes=labels)

In [ ]:
dot_data = tree.export_graphviz(DecisionTree_cl, out_file=None, feature_names=tvectorizer.get_feature_names(),rotate = True, filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph

### **4.2 Iteración 2**


En la siguiente iteración, entrenaremos los siguientes modelos con las características principales seleccionadas:

- Gaussian Classifier
- K Neighbors Classifier
- Decision Tree Classifier
- Random Forest Classifier
- XGB Classifier
- LGBM Classifier


Asimismo, redefiniremos la función de normalización de texto quedándonos sólo con los tokens cuyo part of speach está dentro de las 30 características más importantes.



In [ ]:
# Submuestro y balanceo de clases
data_sample = data.groupby('stars')
data_sample = pd.DataFrame(data_sample.apply(lambda x: x.sample(data_sample.size().min()-25000).reset_index(drop=True))).reset_index(drop=True)
data_train = data_sample.copy()

In [ ]:
data_train.shape

In [ ]:
def dataCleaning(sentence):
    doc = nlp(sentence)
    tokens = []
    for token in doc:
        if len(token)>1: #si el token tiene más de 1 caracteres
            # Forma base del token, sin sufijos de flexión. Y lo pasamos a minuscula.
            if token.lemma_ != '-PRON-' and (token.pos_ == 'ADV' or token.pos_ == 'ADJ' or token.pos_ == 'VERB' or token.pos_ == 'PROPN' or token.pos_ == 'NOUN'): 
              temp = token.lemma_.lower()
              tokens.append(temp)
              clean_tokens = []
              # Quitamos stopswords
              for token in tokens:
                  #if token not in punct and token not in stopwords:
                  if token not in stopwords:
                      clean_tokens.append(token)
    return clean_tokens

def remove_accented_chars(text):
    # Removemos los caracteres especiales
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Eliminamos cualquier caracter que no sen los siguientes: a-z A-Z 0-9   
    pattern = r'[^a-zA-Z0-9\s]' 
    text = re.sub(pattern, '', text)
     
    return text

In [ ]:
titular_list_clean=[]

i=0
titular_clean=[]
for titular in data_train['review_all']:
    titular=remove_accented_chars(str(titular))
    titular_clean=dataCleaning(titular)
    titular_clean=' '.join(titular_clean)
    titular_list_clean.append(titular_clean)
    i=+1

In [ ]:
result = pd.Series(titular_list_clean)

In [ ]:
len(result)

In [ ]:
X_modelo = pd.DataFrame(matriz_titulos_count_tvectorizer.toarray(), columns=tvectorizer.get_feature_names())
X_modelo.head(2)

In [ ]:
Gaussian_cl = GaussianNB().fit(X_train,y_train)
KNeighbors_cl = KNeighborsClassifier().fit(X_train,y_train)
DecisionTree_cl = DecisionTreeClassifier().fit(X_train,y_train)
RandomForest_cl = RandomForestClassifier().fit(X_train,y_train)
XGB_cl = XGBClassifier().fit(X_train,y_train)
LGBM_cl= LGBMClassifier().fit(X_train,y_train)

In [ ]:
labels = [1, 2, 3, 4, 5]
# Binarize ytest with shape (n_samples, n_classes)
y_testb = label_binarize(y_test, classes=labels)

In [ ]:
modelos = ['Gaussian Classifier', 'K Neighbors Classifier', 'Decision Tree Classifier', 'Random Forest Classifier', 'XGB Classifier', 'LGBM Classifier']

for i, model in enumerate([Gaussian_cl, KNeighbors_cl, DecisionTree_cl, RandomForest_cl, XGB_cl, LGBM_cl]):
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    # Binarize ypreds with shape (n_samples, n_classes)
    y_train_preds = label_binarize(y_train_pred, classes=labels)
    y_test_preds = label_binarize(y_test_pred, classes=labels)
        
    print(f'Modelo: {modelos[i]}')
    print('ROC AUC Train', roc_auc_score(y_train,y_train_preds, multi_class='ovr'))
    print('ROC AUC Test', roc_auc_score(y_test,y_test_preds, multi_class='ovr'))
    metrics.plot_confusion_matrix(model, X_test, y_test, values_format = '.0f')
    plt.show()
    print('\n')

De los modelos construidos en la presente iteración, los modelos de árboles de clasificación son lo únicos que han podido predecir mejor las clases extremas (1 estrella y 5 estrella) y presentado un menor error en la predicción de las clases ambiguas (2, 3 y 4 estrellas). Sin embaego, nuestro modelo de benchmark junto con el XGB Classifier son los únicos que no han presentando indicios de overfiting. Por su mejor performance en tiempos de ejecución y en rendimientos de la métrica elegida, es que continuaremos trabajando con el LGBM Classifier.

### **4.3 Iteracion 3**


Por los motivos anteriormente enunciadas, no podremos hacer una búsqueda exhautiva de hiperparámetros con técnicas como Grid Search, ya que por la cantidad de features la ejecución se demora más de los límites de tiempos permitidos por Colab.

En la presente iteración realizaremos una optimización de hiperparámetros con Random Search aplicado k fold validation, cuya técnica nos permite recorrer una mayor rango de posibilidades a una cantidad de iteracion prefijadas.

In [ ]:
param_trees = {'n_estimators': [100, 150, 200, 250],  # Número de árboles 
               'metodo': ['SVD', 'KBEST'],
               'max_depth': [10, 15, 20, 50, 60,-1],  # Profundidad
               'num_leaves': [7, 14, 21, 30, 50, 60], # Máximo de hojas de árboles
               'min_child_samples':[15, 20, 30, -1],  # Número mínimo de datos necesarios en un niño (hoja)
               }

roc_auc_ovr_scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')

# ESTRATEGIA 2: Random Search
model = LGBMClassifier(random_state=42, subsample=0.7)
rs = RandomizedSearchCV(model, param_trees, n_iter=50, scoring=roc_auc_ovr_scorer, verbose=2 , n_jobs=3)
rs.fit(X_train, y_train)

In [ ]:
print("Mejores parametros: "+str(rs.best_params_))
print("Mejor Score: "+str(rs.best_score_)+'\n')

scores_2 = pd.DataFrame(rs.cv_results_)
scores_2

In [ ]:
LGBM_clf = LGBMClassifier(n_estimators=250, metodo='KBEST', min_child_samples=30, num_leaves=30, random_state=42)
LGBM_clf.fit(X_train, y_train)

In [ ]:
y_train_pred = LGBM_clf.predict(X_train)
y_test_pred = LGBM_clf.predict(X_test)

# Binarize ypreds with shape (n_samples, n_classes)
y_train_preds = label_binarize(y_train_pred, classes=labels)
y_test_preds = label_binarize(y_test_pred, classes=labels)
        
print('ROC AUC Train', roc_auc_score(y_train,y_train_preds, multi_class='ovr'))
print('ROC AUC Test', roc_auc_score(y_test,y_test_preds, multi_class='ovr'))
metrics.plot_confusion_matrix(LGBM_clf, X_test, y_test, values_format = '.0f')
plt.show()

Con nuestra optimización, obtenemos una performace muy parecida a la presentada previamente. Si bien no se ha logrdo mejorar el rendimiento de la clasificación, nos aseguramos de que nuestros resultados son estables y no producto del azar.

## Investigacion: 

En el apartado anterior pudimos observar que en el mejor de los casos, logramos construir un modelo que puede diferenciar una review de 1 estrella a una de 5 estrellas, pero que no logra clasificar adecuadamente aquellas reviews con puntuaciones intermedias de 2, 3 y 4 estrellas.

En este punto nos preguntamos: ¿Valdrá la pena convertir el problema de Machine Learning en un problema binario? Es decir, asignar únicamente las etiquetas Positiva y Negativa a cada crítica y hacer un modelo que, en lugar de predecir las estrellas, prediga esa etiqueta.

Teniendo en cuenta que una puntuación de 2 a 3 estrellas o de 3 a 4 estrellas puede ser una apreciación muy subjetiva colmada de grises, es que nuestra hipótesis será que un clasificador binario performará mucho mejor en la tarea de predecir si un producto será puntuado como positivo o negativo.

Para realizar esta investigación realizaremos el mismo proceso de Normalización y Vectorización de los datos, tomando sólo aquellas reviews que pertenezcan a alguno de los extremos de nuestro problema:

Reviews Negativas: 1 y 2 estrellas
Reviews Positivas: 4 y 5 estellas.
Las instancias pertenecientes a 3 estrellas serán desestimadas por 2 motivos: el primero, por practicidad para el balanceo de clases y el segundo, porque en el hipotético caso en que el usuario tuviese únicamente la opción de calificar como conforme o inconforme, la situación parcial de 3 estrellas queda sin efecto.

En el presente aparatado, construiremos los mismos clasificadores que en el segmento anterior y evaluaremos su performance con la métrica de Accuracy, ya que es equivalente a ROC AUC para clases balanceadas. Finalmente, optimizatemos hiperparámetros del modelo que obtenga mejor rendimiento.

In [ ]:
# Balanceamos clases y eliminamos el punto medio, cuya existencia no sería tal en un problema de clasificación positiva / negativa:

data_train_sent = data_train[data_train['stars']!=3]

In [ ]:
data_train_sent.stars_calif.value_counts()


In [ ]:
titular_list_clean=[]

i=0
titular_clean=[]
for titular in data_train_sent['review_all']:
    titular=remove_accented_chars(str(titular))
    titular_clean=dataCleaning(titular)
    titular_clean=' '.join(titular_clean)
    titular_list_clean.append(titular_clean)
    i=+1

In [ ]:
result_sent = pd.Series(titular_list_clean)


In [ ]:
result_sent


In [ ]:
# numero minimo y maximo de tokens consecutivos que se consideran
MIN_NGRAMS=1
MAX_NGRAMS=4
# cantidad maxima de docs que tienen que tener a un token para conservarlo.
MAX_DF= 0.8
max_features=1000

In [ ]:
tvectorizer = TfidfVectorizer(lowercase=True, strip_accents='unicode', decode_error='ignore',
                             ngram_range=(MIN_NGRAMS, MAX_NGRAMS), max_df=MAX_DF, max_features=max_features)
matriz_titulos_tfidf_tvectorizer = tvectorizer.fit_transform(result_sent)

In [ ]:
X4 = matriz_titulos_tfidf_tvectorizer.toarray()
Y4 = data_train_sent['stars_calif']

X_train, X_test, y_train, y_test = train_test_split(X4,Y4,test_size=0.3,random_state=42,stratify=Y4)

In [ ]:
Gaussian_cl = GaussianNB().fit(X_train,y_train)
KNeighbors_cl = KNeighborsClassifier().fit(X_train,y_train)
DecisionTree_cl = DecisionTreeClassifier().fit(X_train,y_train)
RandomForest_cl = RandomForestClassifier().fit(X_train,y_train)
XGB_cl = XGBClassifier().fit(X_train,y_train)
LGBM_cl= LGBMClassifier().fit(X_train,y_train)

In [ ]:
modelos = ['Gaussian Classifier', 'K Neighbors Classifier', 'Decision Tree Classifier', 'Random Forest Classifier', 'XGB Classifier', 'LGBM Classifier']

for i, model in enumerate([Gaussian_cl, KNeighbors_cl, DecisionTree_cl, RandomForest_cl, XGB_cl, LGBM_cl]):
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
        
    print(f'Modelo: {modelos[i]}')
    print('Accuracy Train', accuracy_score(y_train,y_train_pred))
    print('Accuracy Test', accuracy_score(y_test,y_test_pred))
    metrics.plot_confusion_matrix(model, X_test, y_test, values_format = '.0f')
    plt.show()
    print('\n')

In [ ]:
param_trees = {'n_estimators': [100, 150, 200, 250],  # Número de árboles 
               'metodo': ['SVD', 'KBEST'],
               'max_depth': [10, 15, 20, 50, 60,-1],  # Profundidad
               'num_leaves': [7, 14, 21, 30, 50, 60], # Máximo de hojas de árboles
               'min_child_samples':[15, 20, 30, -1],  # Número mínimo de datos necesarios en un niño (hoja)
               }

# ESTRATEGIA 2: Random Search
model = LGBMClassifier(random_state=42, subsample=0.7)
rs = RandomizedSearchCV(model, param_trees, n_iter=50, scoring= 'accuracy', verbose=2 , n_jobs=3)
rs.fit(X_train, y_train)

In [ ]:
print("Mejores parametros: "+str(rs.best_params_))
print("Mejor Score: "+str(rs.best_score_)+'\n')

scores_2 = pd.DataFrame(rs.cv_results_)
scores_2

In [ ]:
LGBM_clf = LGBMClassifier(n_estimators=250, metodo='KBEST', min_child_samples=30, num_leaves=30, random_state=42)
LGBM_clf.fit(X_train, y_train)

In [ ]:
y_train_pred = LGBM_clf.predict(X_train)
y_test_pred = LGBM_clf.predict(X_test)
        
print('Accuracy Train', accuracy_score(y_train,y_train_pred))
print('Accuracy Test', accuracy_score(y_test,y_test_pred))
metrics.plot_confusion_matrix(LGBM_clf, X_test, y_test, values_format = '.0f')
plt.show()

## Investigacion (2): Utilizacion de redes neuronales para analisis de sentimientos

Hasta el momento, hemos realizado análisis de tipo descriptivo, haciendo un mapeo de las reviews buscando obtener un primer vistazo de la muestra, y hemos realizado modelos de machine learning para obtener distintas respuestas de predicción de nuestros datos


En este apartado nos planteamos como hipótesis de investigación de trabajo la relevancia de utilizar el análisis de sentimientos y posibles usos de esta técnica para la base de datos de Amazon. Esta tecnica se ha vuelto en una de las predilectas debiido a su amplia versatilidad para buscar obtener la opinion de los distintos usuarios de un determinado servicio mediante relaciones estadísticas y de asociación.

Hay muchos enfoques utilizados para realizar este análisis, en dónde las de mayor popularidad abundan en las técnicas de machine learning como lo son el modelo "Multinomial de Bayes" o de maquinas de soporte vectorial. Sin embargo, de acuerdo a cierta literatura académica, los modelos de Deep Learning son los de mejor performance y precisión; como sostienen Liao (2017), Zhang (2018) y Liu (2015). Esto se puede revisar en los links adjuntos más abajo. 

Links a los trabajos citados:
* Liao (2017): https://www.sciencedirect.com/science/article/pii/S1877050917312103
* Zhang (2018): https://arxiv.org/ftp/arxiv/papers/1801/1801.07883.pdf
* Liu (2015): https://www.cs.uic.edu/~liub/FBS/SentimentAnalysis-and-OpinionMining.pdf


En este última sección, basandonos en toda la literatura mencionada previamented, se procederá a aplicar un modelo de Deep Learning para el análisis de sentimiento. Para ello, realizaremos lo siguiente:



* Carga de datos: Por la manera en que trabajan las librerías de Tensorflow, para este clasificador podremos utilizar la totalidad de las instancias de nuestro dataset de entrenamiento original.
* Eliminaremos las instancias cuya puntuación haya sido de 3 estrellas, manteniendo la lógica ya anunciada.
* Aplicaremos una función de limpieza más sencilla, que nos permita recorrer las 160 mil instancias más rápidamente.
* Aplicaremos Enconding con la función provista por tensorflow datasets
* Realizaremos un proceso de Padding, en el que nos aseguramos que todos los tokens tengan la misma longitud. Este proceso en necesario para poder ser utilizado posteriormente por nuestra red neuronal convolucional.
* Utilizaremos una red neuronal preentrenada, definiremos las funciones que vamos a utlizar y los parámetros a aplicar.
* Finalmente, entrenaremos nuestra red y la evaluaremos, siempre haciendo uso de las funciones de Tensorflow Keras.

In [ ]:
# Importamos Tensor Flow (descargamos la última de google colab)
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers # para las capas de convolución y las capas densas de keras
import tensorflow_datasets as tfds # utilizaremos el tokenizador de tensor flow
from bs4 import BeautifulSoup


In [ ]:
train_data = pd.read_json("/content/gdrive/MyDrive/dataset_es_train.json", lines= True)
train_data.head()

In [ ]:
train_data['stars_calif'] = [1 if  train_data['stars'][i]> 3 else 0 for i in train_data.index]

In [ ]:
# Creo la variable 'review_all', que es una concatenación de 'review_title' y 'review_body'
train_data['review_all']=[(str(train_data['review_title'][i])+" "+str(train_data['review_body'][i])) for i in train_data.index]

In [ ]:
train_data = train_data[train_data['stars']!=3]

In [ ]:
train_data.stars_calif.value_counts()


In [ ]:
data = train_data[['review_all', 'stars_calif']]


In [ ]:
data.shape


In [ ]:
# Defino mi función de limpieza:

def clean_review(review):
    review = BeautifulSoup(review, "lxml").get_text()
    # Eliminamos cualquier caracter que no sen los siguientes: a-z A-Z 0-9 signo de admiración o puntuación y espacios
    review = re.sub(r"[^a-zA-Z0-9!?\"\s]", ' ', review)
    # Eliminamos espacios en blanco adicionales
    review = re.sub(r" +", ' ', review)
    return review

In [ ]:
# Aplico la función review por review y obtenemos nuestro corpus. El corpus es la lista de todo el texto que se quiere analizar
data_clean = [clean_review(review) for review in data.review_all]

In [ ]:
# Defino mi Y
data_labels = data.stars_calif.values

In [ ]:
len(data_clean)


In [ ]:
len(data_labels)

In [ ]:
# Vamos a obtener un vector de números y cada uno de llos representará una palabra diferente.
# Vamos a tokenizar y vectorizar con el corpues de tensorflow. Construye el tokenizador a base de un corpus.

# Definimos el tokenizador:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(data_clean, target_vocab_size=2**16)

# Tokenizamos:
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [ ]:
len(data_inputs)


In [ ]:
# Vamos a querer entrenar por bloques, es decir, por conjuntos de frases. Para esto necesitamos que todos tengan la misma logitud.
# El proceso de pading agrega 0 a cada una de esas frases para que todas tengan la misma longitud. Lo hacemos con 0 porque nuestro tokenizador no asigna ese numero a ninguna palabra.

# La maxima longitud a ser considerada será la longitud de la frase mas larga que tengamos en nuestro dataset
MAX_LEN = max([len(sentence) for sentence in data_inputs])

# El pad_sequences nos sirve para añadir algo al principio o al final de una secuencia
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0, # Asignamos el 0 anteriormente indicado
                                                            padding="post", # Le decimos que sea al final
                                                            maxlen=MAX_LEN) # Le pedimos que tenga la máxima longitud de la frase

In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(data_inputs, data_labels, test_size=0.2, random_state=42, stratify=data_labels)


In [ ]:
# Definimos una clase que hereda de una clase de tensor flow - keras - model
class DCNN(tf.keras.Model):
    # Declaramos el constructor y definimos las capas que van a ser utilizadas
    def __init__(self, #hacemos referencia al propio objeto de la clase que va a guardar los parámetros
                 # Definimos la lista de parámetros que vamos a utilizar para construir nuestro modelo de red neuronal convolucional
                 vocab_size, #tamaño del volabulario
                 emb_dim=128, #dimension de emberdding, a qué espacio vectorial vamos a embeber nuestra información. Le pedimos que cada palabra sea resumida a un espacio vectorial de 128 números
                 nb_filters=50, #cuántos filtros vamos a utilizar en cada palabra para obtener las correlaciones entre ellas
                 FFN_units=512, #numeros de neuronas de la capa oculta
                 nb_classes=2, #categorías de clasificación
                 dropout_rate=0.1, #es para que ciertas neuronas se desactiven y que no todas aprendan a la vez, es para evitar el overfiting. El 10% de las neuronas no transmitiran lo que han aprendido en la fase de entrenamiento
                 training=False, #le indicamos que sólo desactivaremos las neuronas durante la fase de entrenamiento, nunca durante la fase de predicción
                 name="dcnn" #le asignamos un nombre al modelo
                 ):
        # Inicializamos el modelo y hacemos la llamada a la superclase
        super(DCNN, self).__init__(name=name)
        
        # Deinimos la capa de embeding
        self.embedding = layers.Embedding(vocab_size, emb_dim)

        # Definimos 3 familias de filtros de convolución, que van a analizar 2, 3 y 4 palabras:
        self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2, padding="valid", activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters, kernel_size=3, padding="valid", activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4, padding="valid", activation="relu")

        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento así que podemos usar la misma capa para cada paso de pooling

        # Ahora definimos la red neuronal que se va a encargar de la clasificación
        # Definimos la capa densa (la capa oculta)
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")

        # Capa de dropout para prevenir el overfiting
        self.dropout = layers.Dropout(rate=dropout_rate)

        # Capa de salida, última capa densa. La función de activación va a depender de la cantidad de clases a predecir.
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1, activation="sigmoid") # nos va a devolver 0 ó 1
        else:
            self.last_dense = layers.Dense(units=nb_classes, activation="softmax") # nos va a dar las probabilidades reales
    
    # Creamos la función que se va a utilizar para llamar al modelo
    def call(self, inputs, training): # vamos a tener que pasarle las entradas y si estamos o no en la fase de entrenamiento para aplicar el dropout
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        # concatenemos las 4 entradas a la red neuronal
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [ ]:
# Definimos los parámetros globales

VOCAB_SIZE = tokenizer.vocab_size #numero de palabras diferentes que vamos a utilizar

EMB_DIM = 200 # Dimension de embeding. Cada palabra se identiicará con un punto de 200 coordenadas
NB_FILTERS = 100 # Filtros de la red neuronal convolucional
FFN_UNITS = 256 # Numero de unidades que tendrá en la capa oculta
NB_CLASSES = 2 #len(set(train_labels))

DROPOUT_RATE = 0.2 # Tasa de olvido durante la propagación hacia atrás

BATCH_SIZE = 32 # Tamaño del bloque de elementos a entrenar (de 32 en 32 reviews para evitar el overfiting) Es un batch learning
NB_EPOCHS = 5 #Numero de veces que vamos a pasar por todo el conjunto de entrenamiento. Vamos a iterar 5 veces sobre todo el dataset.

In [ ]:
# Creamos la red neuronal convolucional con los parámetros anteriormente definidos
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
# Compilamos la red en función de la cantidad de clases
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) # Nos devuelve qué porcentaje del texto es correctamente predicho por nuestro modelo
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["sparse_categorical_accuracy"]) # Buscamos una colección de números que

In [ ]:
# Ajustamos los datos
history = Dcnn.fit(train_inputs, train_labels, batch_size=BATCH_SIZE, epochs=NB_EPOCHS)
#ckpt_manager.save()

Como se puede ver como resultado del último epoch, tenemos un accuracy mayor un poco mayor al 99.5%, mostrando el nivel de preción que tienen estos modelos, aunque lucen un poco overfitteados.

Arriba podemos plotear a los accuracy de train y de test y las pérdidas de validación.

In [ ]:
import matplotlib.pyplot as plt
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  #plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])


plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
results_train = Dcnn.evaluate(train_inputs, train_labels, batch_size=BATCH_SIZE)
results_test = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results_train)
print(results_test)

### Ejemplos

In [ ]:
Dcnn(np.array([tokenizer.encode("Me llego bien el producto")]), training=False).numpy()


In [ ]:
Dcnn(np.array([tokenizer.encode("El producto es de MMUY buena calidad")]), training=False).numpy()

In [ ]:
Dcnn(np.array([tokenizer.encode("No lo recomiendo no funciona")]), training=False).numpy()

In [ ]:
Dcnn(np.array([tokenizer.encode("Esto es un DESASTRE!!!")]), training=False).numpy()

La performance de el último clasificador obtenido es más que satisfactario. Además de la métrica obtenida, cabe resaltar que la forma en que procesa la infomación tensorflow nos ha permitido procesar la totalidad de las intancias sin inconvenientes de procesamiento, sin necesidad de aplicar una selección de features y sin necesidad de realizar grandes esfuerzos en análisis de limpieza y normalización de los tokens.

**Conclusión**


En el presente trabajo, nos propusimos construir un clasificador que prediga la cantidad de estrellas con las que se calificará a un producto a partir de la crítica escrita en la reseña por el usuario.

Para lograrlo, se aplicaron técnicas de normalización de texto y vectorización, se probaron diferentes modelos y se realizó una busqueda de hiperparámetros óptimos. Los resultados obtenidos en la predicción de estrellas no lograron superar significativamente los obtenidos al azar, sin embargo, han obtenido grandes mejoras convirtiendo nuestra clasificación en un problema binario.

Finalmente, se obtuvieron resultados más satisfactorios aplicado un modelo de deep learning que de machine learning, no sólo por la performance en la clasificación, sino también por el tiempo y capacidad de procesamiento del mismo.

Como futuras lineas de investigacion, se podria trabajar en implementar otros modelos para predicción como el modelo BERT. Además, sería recomendable utilizar datos geospaciales de los clientes para ver posibles posibles tendencias en determinadas regionales, que llevan a un mejor uso de los recursos  